In [1]:
from PIL import Image as Im
from IPython.display import Image
import numpy as np
import pickle
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms


import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [3]:
train_data = unpickle("../data/processed_data/all_cf_batches.p")

with open("../data/processed_data/all_labels.txt") as lab:
    train_labels = [int(i) for i in lab.read().split()]
    
label_names = {
    0:'airplane',
    1:'automobile',
    2:'bird',
    3:'cat',
    4:'deer',
    5: 'dog',
    6: 'frog',
    7: 'horse',
    8: 'ship',
    9: 'truck'
}

In [4]:
train_data.shape

(50000, 32, 32, 3)

In [5]:
# ind = 12411

# print(label_names[Y_train[ind]])
# plt.imshow(X_train[ind]);

In [6]:
train_data, X_test, train_labels, Y_test = train_data[:45000], train_data[45000:], train_labels[:45000], train_labels[45000:]

In [7]:
X_train, X_val, Y_train, Y_val = train_data[:40000], train_data[40000:], train_labels[:40000], train_labels[40000:]

print("Train data:")
print(X_train.shape)
print("="*20)
print("Validation data:")
print(X_val.shape)
print("="*20)
print("Test data:")
print(X_test.shape)

Train data:
(40000, 32, 32, 3)
Validation data:
(5000, 32, 32, 3)
Test data:
(5000, 32, 32, 3)


In [8]:
class Cifar(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels 
        self.transform=transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
#         image = Im.fromarray(image)
        
        if self.transform:
            image = self.transform(image)

        return image, self.labels[idx]

In [9]:
transform = transforms.Compose(
    [transforms.ToTensor()])

train_dataset = Cifar(X_train, Y_train, transform=transform)
test_dataset = Cifar(X_test, Y_test, transform=transform)

dataloader_train = DataLoader(train_dataset, batch_size=8,
                        shuffle=True, num_workers=4)

dataloader_test = DataLoader(test_dataset, batch_size=8,
                        shuffle=True, num_workers=4)


In [11]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 16, 3, padding=1)
        self.maxpool = nn.MaxPool2d(kernel_size=(2, 2))
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv4 = nn.Conv2d(32, 32, 3, padding=1)
        self.fc = nn.Linear(8 * 8 * 32, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.maxpool(x)
        
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))
        x = self.maxpool(x)
        
        x = x.view(-1, 8 * 8 * 32)
        x = self.fc(x)
        return x


In [12]:
net = Net()

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [14]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader_train, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 2.302
[1,   400] loss: 2.300
[1,   600] loss: 2.288
[1,   800] loss: 2.237
[1,  1000] loss: 2.119
[1,  1200] loss: 2.083
[1,  1400] loss: 2.027
[1,  1600] loss: 2.010
[1,  1800] loss: 1.969
[1,  2000] loss: 1.978
[1,  2200] loss: 1.914
[1,  2400] loss: 1.832
[1,  2600] loss: 1.864
[1,  2800] loss: 1.815
[1,  3000] loss: 1.842
[1,  3200] loss: 1.744
[1,  3400] loss: 1.703
[1,  3600] loss: 1.669
[1,  3800] loss: 1.673
[1,  4000] loss: 1.636
[1,  4200] loss: 1.606
[1,  4400] loss: 1.593
[1,  4600] loss: 1.570
[1,  4800] loss: 1.567
[1,  5000] loss: 1.559
[2,   200] loss: 1.529
[2,   400] loss: 1.527
[2,   600] loss: 1.504
[2,   800] loss: 1.541
[2,  1000] loss: 1.478
[2,  1200] loss: 1.457
[2,  1400] loss: 1.501
[2,  1600] loss: 1.460
[2,  1800] loss: 1.447
[2,  2000] loss: 1.453
[2,  2200] loss: 1.440
[2,  2400] loss: 1.417
[2,  2600] loss: 1.446
[2,  2800] loss: 1.391
[2,  3000] loss: 1.377
[2,  3200] loss: 1.432
[2,  3400] loss: 1.348
[2,  3600] loss: 1.406
[2,  3800] 

In [15]:
correct = 0
total = 0
for data in dataloader_train:
    images, labels = data
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 69 %


In [16]:
images.shape

torch.Size([8, 3, 32, 32])